In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import pandas as pd
import random
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from imblearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer, RobustScaler
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split, cross_validate
from imblearn.under_sampling import TomekLinks
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from yellowbrick.classifier.rocauc import roc_auc
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from yellowbrick.classifier import ROCAUC
from imblearn.over_sampling import SMOTENC
from sklearn.metrics import roc_curve, auc
random.seed(2814)

In [2]:
df = pd.read_csv('../data/processed/3_seai_miss_forest_imputation.csv')
df.drop(['BerRating', 'CO2Rating'], axis = 1, inplace=True)
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(0, 'zero')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(1, 'one')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(2, 'two')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(3, 'three')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].replace(4, 'four')
df['NoOfSidesSheltered'] = df['NoOfSidesSheltered'].astype('category')
df['YearofConstruction'] = df['YearofConstruction'].astype('category')

df['EnergyRating'] = df['EnergyRating'].replace('A1', 'A')
df['EnergyRating'] = df['EnergyRating'].replace('A2', 'A')
df['EnergyRating'] = df['EnergyRating'].replace('A3', 'A')
df['EnergyRating'] = df['EnergyRating'].replace('B1', 'B')
df['EnergyRating'] = df['EnergyRating'].replace('B2', 'B')
df['EnergyRating'] = df['EnergyRating'].replace('B3', 'B')
df['EnergyRating'] = df['EnergyRating'].replace('C1', 'C')
df['EnergyRating'] = df['EnergyRating'].replace('C2', 'C')
df['EnergyRating'] = df['EnergyRating'].replace('C3', 'C')
df['EnergyRating'] = df['EnergyRating'].replace('D1', 'D')
df['EnergyRating'] = df['EnergyRating'].replace('D2', 'D')
df['EnergyRating'] = df['EnergyRating'].replace('E1', 'E')
df['EnergyRating'] = df['EnergyRating'].replace('E2', 'E')
df['EnergyRating'] = df['EnergyRating'].replace('F', 'F')
df['EnergyRating'] = df['EnergyRating'].replace('G', 'G') 

In [3]:
new_cols = [col for col in df.columns if col != 'EnergyRating'] + ['EnergyRating']
df = df[new_cols]
del(new_cols)

In [4]:
X = df.iloc[:, :-1] # Independent Variables
y = df.iloc[:, -1] # Dependent Variables

In [5]:
numerical = [
    'GroundFloorArea(sq m)'
    ,'TotalDeliveredEnergy'
    ,'InsulationThickness'
]

X[numerical] = MinMaxScaler().fit_transform(X[numerical])

In [6]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y,  random_state=2814)

In [9]:
X_train

,CountyName,DwellingTypeDescr,YearofConstruction,GroundFloorArea(sq m),MainSpaceHeatingFuel,MainWaterHeatingFuel,VentilationMethod,StructureType,NoOfSidesSheltered,InsulationType,InsulationThickness,TotalDeliveredEnergy
243567,Donegal,House,2002,0.041128,Anthracite,Anthracite,Natural vent.,Timber or Steel Frame,two,Factory Insulated,0.016026,0.001022
616016,Dublin,House,1930,0.038657,Electricity,Heating Oil,Natural vent.,Masonry,two,Loose Jacket,0.000000,0.011529
346503,Kerry,Semi-detached house,2005,0.032150,Electricity,Electricity,Natural vent.,Timber or Steel Frame,two,Factory Insulated,0.018697,0.003486
610895,Dublin,Semi-detached house,2002,0.046305,Mains Gas,Mains Gas,Natural vent.,Masonry,three,Factory Insulated,0.022650,0.004352
1004118,Wexford,Detached house,1960,0.050612,Heating Oil,Electricity,Natural vent.,Masonry,two,None,0.000000,0.010839
...,...,...,...,...,...,...,...,...,...,...,...,...
703486,Louth,End of terrace house,1997,0.023874,Mains Gas,Mains Gas,Natural vent.,Masonry,one,Loose Jacket,0.042735,0.002967
445063,Donegal,Semi-detached house,2003,0.031763,Heating Oil,Heating Oil,Natural vent.,Masonry,two,Factory Insulated,0.021154,0.003629
32751,Tipperary,Mid-terrace house,1975,0.027580,Solid Multi-Fuel,Solid Multi-Fuel,Natural vent.,Masonry,two,Factory Insulated,0.000000,0.004894
996162,Kerry,House,2008,0.039001,Heating Oil,Heating Oil,Natural vent.,Masonry,zero,Factory Insulated,0.000000,0.004486


In [10]:
# Create the oversampler
smotenc = SMOTENC([0,1,2,4,5,6,7,8,9],random_state = 2814)
X_train, y_train = smotenc.fit_resample(X_train, y_train)

In [11]:
# https://stackoverflow.com/a/52935270/5923619
# One Hot Encodes our categorical feature and binds it to the original dataset
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [12]:
# One Hot Encode all of our categorical features
categorical = [ 
    'YearofConstruction'
    ,'CountyName'
    ,'DwellingTypeDescr'
    ,'MainSpaceHeatingFuel'
    ,'MainWaterHeatingFuel'
    ,'VentilationMethod'
    ,'StructureType'
    ,'InsulationType'
    ,'NoOfSidesSheltered'
]

for feature in categorical:
    X_train = encode_and_bind(X_train, feature)

In [13]:
# One Hot Encode all of our categorical features
for feature in categorical:
    X_test = encode_and_bind(X_test, feature)

# Define SMOTE-Tomek Links
resample=SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'), smote=SMOTE(k_neighbors=6), random_state=2814)
X_train, y_train = resample.fit_resample(X_train, y_train)

In [14]:
X_test = X_test.drop('MainWaterHeatingFuel_Electricity - On-peak Night-Ra', axis=1)
X_train = X_train.drop('MainSpaceHeatingFuel_Bioethanol from renewable sour', axis=1)
X_train = X_train.drop('MainSpaceHeatingFuel_Biodiesel from renewable sourc', axis=1)
X_train = X_train.drop('MainWaterHeatingFuel_None', axis=1)

In [17]:
X_train.to_csv('../data/interim/1_X_train.csv', index=False)
y_train.to_csv('../data/interim/1_y_train.csv', index=False)
X_test.to_csv('../data/interim/1_X_test.csv', index=False)
y_test.to_csv('../data/interim/1_y_test.csv', index=False)

In [15]:
# Modelling
dt = DecisionTreeClassifier(random_state=2814)
dt.fit(X_train, y_train)

y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

          A1       0.49      0.77      0.59       252
          A2       0.91      0.91      0.91      8648
          A3       0.91      0.89      0.90     10239
          B1       0.60      0.73      0.66      3052
          B2       0.64      0.72      0.68      6563
          B3       0.78      0.75      0.77     15594
          C1       0.80      0.78      0.79     22757
          C2       0.78      0.77      0.78     24906
          C3       0.77      0.77      0.77     23637
          D1       0.80      0.79      0.80     22871
          D2       0.83      0.82      0.82     19637
          E1       0.77      0.79      0.78     11326
          E2       0.78      0.80      0.79      8956
           F       0.83      0.85      0.84      9269
           G       0.95      0.94      0.95     13363

    accuracy                           0.80    201070
   macro avg       0.78      0.81      0.79    201070
weighted avg       0.81   

In [16]:
rf = RandomForestClassifier(random_state=2814)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

KeyboardInterrupt: 

In [ ]:
lr = LogisticRegression(random_state=2814, solver='sag')
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

/Users/oisinbrannock/.pyenv/versions/3.10.5/envs/masters_thesis/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

          A1       0.03      0.70      0.07       314
          A2       0.59      0.48      0.53     10810
          A3       0.53      0.48      0.51     12798
          B1       0.05      0.21      0.08      3815
          B2       0.12      0.33      0.17      8203
          B3       0.19      0.24      0.21     19493
          C1       0.24      0.25      0.24     28446
          C2       0.23      0.15      0.18     31133
          C3       0.22      0.12      0.15     29546
          D1       0.23      0.17      0.20     28588
          D2       0.21      0.12      0.16     24547
          E1       0.15      0.10      0.12     14158
          E2       0.13      0.14      0.13     11195
           F       0.17      0.29      0.22     11587
           G       0.51      0.61      0.56     16704

    accuracy                           0.24    251337
   macro avg       0.24      0.29      0.23    251337
weighted avg       0.26   

In [ ]:
svm = LinearSVC(random_state=2814)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print(classification_report(y_test, y_pred))